In [11]:
import numpy as np
import pandas as pd
print(np.__version__)
print(pd.__version__)


1.26.4
2.2.2


In [13]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd

In [15]:
pip install tensorflow

In [16]:
pip show tensorflow

Name: tensorflow
Version: 2.19.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\Acer\anaconda3\Lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, termcolor, typing-extensions, wrapt
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [24]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


TensorFlow version: 2.19.0
Num GPUs Available: 0


In [25]:
import tensorflow as tf

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [27]:
def create_data_set(path):
    images = []
    labels = []
    folders = os.listdir(path)
    for folder in folders:
        folder_path = os.path.join(path , folder)
        items = os.listdir(folder_path)
        for item in items:
            file_path = os.path.join(folder_path , item)
            images.append(file_path)
            labels.append(folder)
    df = pd.DataFrame({"file_path": images, "labels": labels})
    return df   # <- return it

In [32]:
waste_df = create_data_set(r"C:\Users\Acer\OneDrive\Desktop\Green Skills\Waste Classification using CV\garbage-dataset")
waste_df

,file_path,labels
0,C:\Users\Acer\OneDrive\Desktop\Green Skills\Wa...,battery
1,C:\Users\Acer\OneDrive\Desktop\Green Skills\Wa...,battery
2,C:\Users\Acer\OneDrive\Desktop\Green Skills\Wa...,battery
3,C:\Users\Acer\OneDrive\Desktop\Green Skills\Wa...,battery
4,C:\Users\Acer\OneDrive\Desktop\Green Skills\Wa...,battery
...,...,...
19757,C:\Users\Acer\OneDrive\Desktop\Green Skills\Wa...,trash
19758,C:\Users\Acer\OneDrive\Desktop\Green Skills\Wa...,trash
19759,C:\Users\Acer\OneDrive\Desktop\Green Skills\Wa...,trash
19760,C:\Users\Acer\OneDrive\Desktop\Green Skills\Wa...,trash


In [34]:
waste_df.head()

,file_path,labels
0,C:\Users\Acer\OneDrive\Desktop\Green Skills\Wa...,battery
1,C:\Users\Acer\OneDrive\Desktop\Green Skills\Wa...,battery
2,C:\Users\Acer\OneDrive\Desktop\Green Skills\Wa...,battery
3,C:\Users\Acer\OneDrive\Desktop\Green Skills\Wa...,battery
4,C:\Users\Acer\OneDrive\Desktop\Green Skills\Wa...,battery


In [36]:
waste_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19762 entries, 0 to 19761
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_path  19762 non-null  object
 1   labels     19762 non-null  object
dtypes: object(2)
memory usage: 308.9+ KB


In [38]:
waste_df.isna().sum()

file_path    0
labels       0
dtype: int64

In [40]:
from sklearn.model_selection import train_test_split
train_data, test_data= train_test_split(waste_df, test_size =0.2 ,random_state =42)

In [42]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 0.1/255, shear_range = 0.2, zoom_range=0.2, horizontal_flip=True, rotation_range = 45, vertical_flip = True, fill_mode = 'nearest')

In [44]:
test_datagen = ImageDataGenerator(rescale = 1.0/255)

In [46]:
train_generator = train_datagen.flow_from_dataframe(dataframe = train_data, x_col="file_path", y_col = "labels", target_size = (255,255), batch_size = 32, class_mode = 'categorical')

Found 15809 validated image filenames belonging to 10 classes.


In [47]:
num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)

Number of classes: 10


In [48]:
from  tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Input
from tensorflow.keras.models import Sequential
waste_img_model = Sequential()
waste_img_model.add(Input(shape=(255,255,3)))
waste_img_model.add(Conv2D(32, kernel_size = 3, activation = 'relu'))
waste_img_model.add(MaxPool2D(pool_size = 2))
waste_img_model.add(Conv2D(64, kernel_size = 3, activation = 'relu'))
waste_img_model.add(MaxPool2D(pool_size = 2))
waste_img_model.add(Conv2D(128, kernel_size = 3, activation = 'relu'))
waste_img_model.add(MaxPool2D(pool_size = 2))
waste_img_model.add(Flatten())
waste_img_model.add(Dense(128, activation='relu'))
waste_img_model.add(Dense(num_classes, activation='softmax'))

In [52]:
waste_img_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 253, 253, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 126, 126, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 124, 124, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      14,745,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,840,266 (56.61 MB)

 Trainable params: 14,840,266 (56.61 MB)

 Non-trainable params: 0 (0.00 B)

In [54]:
waste_df.columns

Index(['file_path', 'labels'], dtype='object')

In [65]:
IMG_SIZE = 255

In [67]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    x_col="file_path",
    y_col="labels",
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 3953 validated image filenames belonging to 10 classes.


In [71]:
waste_img_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [75]:
import pickle
# Train
history = waste_img_model.fit(train_generator, validation_data=test_generator, epochs=15)

# Save model properly
waste_img_model.save("waste_classifier_cnn.h5")

# Save class labels separately
with open("class_labels.pkl", "wb") as f:
    pickle.dump(train_generator.class_indices, f)


Epoch 1/15
495/495 ━━━━━━━━━━━━━━━━━━━━ 1461s 3s/step - accuracy: 0.7081 - loss: 0.8589 - val_accuracy: 0.1027 - val_loss: 96.2570
Epoch 2/15
495/495 ━━━━━━━━━━━━━━━━━━━━ 1307s 3s/step - accuracy: 0.7129 - loss: 0.8417 - val_accuracy: 0.0964 - val_loss: 66.4522
Epoch 3/15
495/495 ━━━━━━━━━━━━━━━━━━━━ 1380s 3s/step - accuracy: 0.7121 - loss: 0.8365 - val_accuracy: 0.0933 - val_loss: 86.1261
Epoch 4/15
495/495 ━━━━━━━━━━━━━━━━━━━━ 1256s 3s/step - accuracy: 0.7174 - loss: 0.8194 - val_accuracy: 0.1045 - val_loss: 82.9582
Epoch 5/15
495/495 ━━━━━━━━━━━━━━━━━━━━ 1251s 2s/step - accuracy: 0.7241 - loss: 0.8165 - val_accuracy: 0.0807 - val_loss: 108.7392
Epoch 6/15
495/495 ━━━━━━━━━━━━━━━━━━━━ 1139s 2s/step - accuracy: 0.7324 - loss: 0.7838 - val_accuracy: 0.1022 - val_loss: 115.0143
Epoch 7/15
495/495 ━━━━━━━━━━━━━━━━━━━━ 1156s 2s/step - accuracy: 0.7366 - loss: 0.7781 - val_accuracy: 0.0999 - val_loss: 106.0618
Epoch 8/15
495/495 ━━━━━━━━━━━━━━━━━━━━ 1162s 2s/step - accuracy: 0.7369 - loss:

In [77]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predict on test set
y_pred = waste_img_model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

# Print report
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred_classes, target_names=list(test_generator.class_indices.keys())))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
print("\nConfusion Matrix:\n")
print(cm)


124/124 ━━━━━━━━━━━━━━━━━━━━ 58s 461ms/step

Classification Report:

              precision    recall  f1-score   support

     battery       0.25      0.03      0.05       164
  biological       1.00      0.01      0.01       176
   cardboard       0.30      0.06      0.09       395
     clothes       0.33      0.00      0.00      1089
       glass       1.00      0.00      0.01       596
       metal       0.00      0.00      0.00       202
       paper       0.02      0.00      0.01       344
     plastic       0.11      0.78      0.19       413
       shoes       0.00      0.00      0.00       390
       trash       0.04      0.18      0.07       184

    accuracy                           0.10      3953
   macro avg       0.31      0.11      0.04      3953
weighted avg       0.34      0.10      0.04      3953


Confusion Matrix:

[[  5   0   2   0   0   0   1 147   0   9]
 [  0   1  14   0   0   0   0 120   0  41]
 [  0   0  22   0   0   0   0 324   0  49]
 [  3   0   9   1   0  

C:\Users\Acer\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Acer\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Acer\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [84]:
# Evaluate model
loss, acc = waste_img_model.evaluate(test_generator)
print(f"✅ Test Accuracy: {acc*100:.2f}%")
print(f"✅ Test Loss: {loss:.4f}")

124/124 ━━━━━━━━━━━━━━━━━━━━ 48s 387ms/step - accuracy: 0.0979 - loss: 133.6345
✅ Test Accuracy: 9.79%
✅ Test Loss: 133.6345


In [100]:
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 6s 1us/step


In [108]:
num_classes = len(train_generator.class_indices)

# Build Model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [110]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [118]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image Data Generators with validation split
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    validation_split=0.2   # reserve 20% for validation
)

# Training generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,             # your train split DataFrame
    x_col="file_path",
    y_col="labels",
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

# Validation generator
val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,             # same DataFrame but validation split
    x_col="file_path",
    y_col="labels",
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical",
    subset="validation",
    shuffle=True
)

print("✅ Classes:", train_generator.class_indices)


Found 12648 validated image filenames belonging to 10 classes.
Found 3161 validated image filenames belonging to 10 classes.
✅ Classes: {'battery': 0, 'biological': 1, 'cardboard': 2, 'clothes': 3, 'glass': 4, 'metal': 5, 'paper': 6, 'plastic': 7, 'shoes': 8, 'trash': 9}


In [120]:
epochs = 20   # try 30–50 for better accuracy if dataset is large
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)


C:\Users\Acer\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
396/396 ━━━━━━━━━━━━━━━━━━━━ 731s 2s/step - accuracy: 0.6371 - loss: 1.1284 - val_accuracy: 0.8323 - val_loss: 0.5261
Epoch 2/20
396/396 ━━━━━━━━━━━━━━━━━━━━ 614s 2s/step - accuracy: 0.8027 - loss: 0.6007 - val_accuracy: 0.8684 - val_loss: 0.4244
Epoch 3/20
396/396 ━━━━━━━━━━━━━━━━━━━━ 572s 1s/step - accuracy: 0.8340 - loss: 0.5058 - val_accuracy: 0.8741 - val_loss: 0.3948
Epoch 4/20
396/396 ━━━━━━━━━━━━━━━━━━━━ 598s 2s/step - accuracy: 0.8556 - loss: 0.4429 - val_accuracy: 0.8817 - val_loss: 0.3676
Epoch 5/20
396/396 ━━━━━━━━━━━━━━━━━━━━ 594s 1s/step - accuracy: 0.8657 - loss: 0.4047 - val_accuracy: 0.8915 - val_loss: 0.3493
Epoch 6/20
396/396 ━━━━━━━━━━━━━━━━━━━━ 593s 1s/step - accuracy: 0.8731 - loss: 0.3858 - val_accuracy: 0.8880 - val_loss: 0.3514
Epoch 7/20
396/396 ━━━━━━━━━━━━━━━━━━━━ 588s 1s/step - accuracy: 0.8751 - loss: 0.3736 - val_accuracy: 0.8890 - val_loss: 0.3420
Epoch 8/20
396/396 ━━━━━━━━━━━━━━━━━━━━ 579s 1s/step - accuracy: 0.8807 - loss: 0.3600 - val_accu

In [121]:
# ✅ Evaluate on test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")


124/124 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.9244 - loss: 0.2274
Test Accuracy: 92.44%
Test Loss: 0.2274


In [124]:
model.save("waste_classifier.h5")


In [126]:
import pickle
with open("class_labels.pkl", "wb") as f:
    pickle.dump(train_generator.class_indices, f)


In [128]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)


In [132]:
print(train_generator.class_indices)



{'battery': 0, 'biological': 1, 'cardboard': 2, 'clothes': 3, 'glass': 4, 'metal': 5, 'paper': 6, 'plastic': 7, 'shoes': 8, 'trash': 9}


In [138]:
class_names = list(train_generator.class_indices.keys())
print("Class names:", class_names)


Class names: ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']


In [142]:
import pickle

# Save history after training
with open("training_history.pkl", "wb") as f:
    pickle.dump(history.history, f)
